# SQL Python

Tenemos 3 CSV, debemos crear una base de datos en Dbeaver y usando python ingresar los datos y después realizarnos 5 preguntas al respecto que podamos sacar con consultas que lleven JOIN, GROUP BY o similar

También podemos practicar subsconsultas

In [1]:
# importamos las librerías con las que vamos a trabajar

# Trabajar con bases de datos y python
# -----------------------------------------------------------------------
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors


# Trabajar con DataFrames
# -----------------------------------------------------------------------
import pandas as pd

from geopy.geocoders import Nominatim
from src.SupportLab import obtain_sky_state,obtain_date_index,obtain_municipio_index,geopy_search

Primero voy a cargar los CSV con objetivo de ver que tablas pueda sacar

In [2]:
foursquare = pd.read_csv("datos/api_foursquare.csv",index_col="Unnamed: 0")
aemet = pd.read_csv("datos/df_aemet_final.csv")
municipios = pd.read_csv("datos/municipios.csv",index_col="Unnamed: 0")

In [3]:
foursquare.head(1)

,municipio,category,fsq_id,closed_bucket,distance,link,name,address,latitude,longitude
0,ajalvir,Park,4d3c6c5b84d46ea87dddfd5c,LikelyOpen,563.0,/v3/places/4d3c6c5b84d46ea87dddfd5c,Diverjungla,"Calle Segovia, 20 (Pol. Ind. Donada), 28864 Aj...",40.529921,-3.484341


In [4]:
foursquare["category"].unique()

array(['Park', 'Monument', 'TV Station', 'Castle', 'Film Studio',
       'Palace', 'Fountain', 'Bridge', 'Beach',
       'Print, TV, Radio and Outdoor Advertising Service'], dtype=object)

In [5]:
aemet.head(1)

,fecha,cielo,temp._(°c),sen._térmica_(°c),racha_máx._(km/h),precipitación_(mm),nieve_(mm),humedad_relativa_(%),prob._precip.__(%),prob._de_nieve_(%),prob._de_tormenta_(%),avisos,dirección_viento,velocidad_del_viento,localizacion_id
0,2024-08-29 12:00:00,Nubes altas,21,21,25.0,0.0,0,69,95,0,75,Riesgo,S,7,acebeda-la-id28001


In [6]:
aemet["localizacion_id"].unique()

array(['acebeda-la-id28001', 'ajalvir-id28002',
       'alameda-del-valle-id28003', 'alamo-el-id28004',
       'alcala-de-henares-id28005', 'alcobendas-id28006',
       'alcorcon-id28007', 'aldea-del-fresno-id28008', 'algete-id28009',
       'alpedrete-id28010', 'ambite-id28011', 'anchuelo-id28012',
       'aranjuez-id28013', 'arganda-del-rey-id28014',
       'arroyomolinos-id28015', 'atazar-el-id28016', 'batres-id28017',
       'becerril-de-la-sierra-id28018', 'belmonte-de-tajo-id28019',
       'berrueco-el-id28021', 'berzosa-del-lozoya-id28020',
       'boadilla-del-monte-id28022', 'boalo-el-id28023',
       'braojos-id28024', 'brea-de-tajo-id28025', 'brunete-id28026',
       'buitrago-del-lozoya-id28027', 'bustarviejo-id28028',
       'cabanillas-de-la-sierra-id28029', 'cabrera-la-id28030',
       'cadalso-de-los-vidrios-id28031', 'camarma-de-esteruelas-id28032',
       'campo-real-id28033', 'canencia-id28034', 'carabana-id28035',
       'casarrubuelos-id28036', 'cenicientos-id28037

In [7]:
aemet["fecha"].unique()

array(['2024-08-29 12:00:00', '2024-08-29 13:00:00',
       '2024-08-29 14:00:00', '2024-08-29 15:00:00',
       '2024-08-29 16:00:00', '2024-08-29 17:00:00',
       '2024-08-29 18:00:00', '2024-08-29 19:00:00',
       '2024-08-29 20:00:00', '2024-08-29 21:00:00',
       '2024-08-29 22:00:00', '2024-08-29 23:00:00',
       '2024-08-30 00:00:00', '2024-08-30 01:00:00',
       '2024-08-30 02:00:00', '2024-08-30 03:00:00',
       '2024-08-30 04:00:00', '2024-08-30 05:00:00',
       '2024-08-30 06:00:00', '2024-08-30 07:00:00',
       '2024-08-30 08:00:00', '2024-08-30 09:00:00',
       '2024-08-30 10:00:00', '2024-08-30 11:00:00',
       '2024-08-30 12:00:00', '2024-08-30 13:00:00',
       '2024-08-30 14:00:00', '2024-08-30 15:00:00',
       '2024-08-30 16:00:00', '2024-08-30 17:00:00',
       '2024-08-30 18:00:00', '2024-08-30 19:00:00',
       '2024-08-30 20:00:00', '2024-08-30 21:00:00',
       '2024-08-30 22:00:00', '2024-08-30 23:00:00',
       '2024-08-31 00:00:00', '2024-08-31 01:0

In [8]:
aemet["cielo"].unique()

array(['Nubes altas', 'Muy nuboso',
       'Cubierto con tormenta y lluvia escasa', 'Nuboso',
       'Cielo despejado', 'Poco nuboso',
       'Intervalos nubosos con lluvia escasa', 'Cubierto',
       'Cubierto con lluvia escasa', 'Cubierto con lluvia',
       'Muy nuboso con tormenta y lluvia escasa', 'Muy nuboso con lluvia',
       'Intervalos nubosos', 'Muy nuboso con lluvia escasa',
       'Nuboso con tormenta y lluvia escasa', 'Cubierto con tormenta',
       'Nuboso con lluvia escasa', 'Intervalos nubosos con lluvia',
       'Bruma', 'Niebla', 'Nuboso con lluvia', 'Muy nuboso con tormenta',
       'Nuboso con tormenta'], dtype=object)

In [9]:
municipios.head(1)

,acebeda-la,ajalvir,alameda-del-valle,alamo-el,alcala-de-henares,alcobendas,alcorcon,aldea-del-fresno,algete,alpedrete,...,villamanta,villamantilla,villanueva-de-la-canada,villanueva-del-pardillo,villanueva-de-perales,villar-del-olmo,villarejo-de-salvanes,villaviciosa-de-odon,villavieja-del-lozoya,zarzalejo
0,41.0869583,40.5342302,40.9177178,40.2307296,40.4818396,40.5400082,40.3492998,40.3215212,40.5961652,40.6599005,...,40.2990939,40.3421033,40.4472106,40.4949232,40.3466066,40.3364465,40.1683582,40.3573787,41.0063622,40.5483016


In [10]:
municipios.T

,0,1,2
acebeda-la,41.0869583,-3.624399,La Acebeda
ajalvir,40.5342302,-3.4807818,Ajalvir
alameda-del-valle,40.9177178,-3.8438216,Alameda del Valle
alamo-el,40.2307296,-3.9905893,El Álamo
alcala-de-henares,40.4818396,-3.3644973,Alcalá de Henares
...,...,...,...
villar-del-olmo,40.3364465,-3.2355624,Villar del Olmo
villarejo-de-salvanes,40.1683582,-3.2738764,Villarejo de Salvanés
villaviciosa-de-odon,40.3573787,-3.9002334,Villaviciosa de Odón
villavieja-del-lozoya,41.0063622,-3.6716291,Villavieja del Lozoya


# Que tablas voy a hacer y por orden:
- Municipios
    - id_municipio PK
    - nombre
    - latitud
    - longitud

- Categorías
    - id_categoria PK
    - tipo

- Estado cielo
    - id_estado PK
    - estado

- Fecha
    - id_fecha PK
    - Fecha

Tiempo
    - id_tiempo PK
    - id_estado_cielo (FK estado cielo)
    - id_municipio (FK municipio)
    - id_fecha (FK fecha)
    - temperatura
    - sensacion_temp
    - racha_max
    - precipitación
    - nieve
    - humedad
    - probabilidad precipitación
    - probabilidad nieve
    - probabilidad tormenta
    - Dirección viento

- Lugares
    - id_lugar PK
    - id_categoría (FK categorías)
    - id_municipio (FK municipio)
    - id_tiempo (FK tiempo)
    - nombre
    - dirección
    - distancia (al centro del municipio)


Ahora tengo que hacer los DF de cada una


# Municipios
- Municipios
    - id_municipio PK
    - nombre
    - latitud
    - longitud


In [11]:
df_municipios = municipios.T

In [12]:
df_municipios.head(1)

,0,1,2
acebeda-la,41.0869583,-3.624399,La Acebeda


In [13]:
df_municipios = df_municipios.rename(columns={0 : "latitud",
                                              1 : "longitud",
                                              2 : "municipio"})
df_municipios
df_municipios.reset_index(inplace=True,drop=True)
df_municipios.head(1)

,latitud,longitud,municipio
0,41.0869583,-3.624399,La Acebeda


In [14]:
df_municipios = df_municipios[["municipio","latitud","longitud"]]
df_municipios.head(1)

,municipio,latitud,longitud
0,La Acebeda,41.0869583,-3.624399


In [15]:
df_municipios["latitud"] = df_municipios["latitud"].astype(float)
df_municipios["longitud"] = df_municipios["longitud"].astype(float)
df_municipios.head(1)

,municipio,latitud,longitud
0,La Acebeda,41.086958,-3.624399


# Categorías
- Categorías
    - id_categoria PK
    - tipo


In [16]:
foursquare.head(1)

,municipio,category,fsq_id,closed_bucket,distance,link,name,address,latitude,longitude
0,ajalvir,Park,4d3c6c5b84d46ea87dddfd5c,LikelyOpen,563.0,/v3/places/4d3c6c5b84d46ea87dddfd5c,Diverjungla,"Calle Segovia, 20 (Pol. Ind. Donada), 28864 Aj...",40.529921,-3.484341


In [17]:
df_categorias = pd.DataFrame(foursquare["category"].unique())
df_categorias = df_categorias.rename(columns={0 : "categoria"})
df_categorias.sample()

,categoria
0,Park


# Estado Cielo
- Estado cielo
    - id_estado PK
    - estado


In [18]:
df_cielo = pd.DataFrame(aemet["cielo"].unique())
df_cielo = df_cielo.rename(columns={0 : "estado_cielo"})
df_cielo.sample()

,estado_cielo
7,Cubierto


In [19]:
df_cielo.to_csv("datos/output/estado_cielo.csv")

# Fecha
- Fecha
    - id_fecha PK
    - Fecha


In [20]:
df_fechas = pd.DataFrame(aemet["fecha"].unique())
df_fechas = df_fechas.rename(columns={0 : "fecha"})
df_fechas["fecha"] = pd.to_datetime(df_fechas["fecha"])
df_fechas.sample()

,fecha
1,2024-08-29 13:00:00


In [21]:
df_fechas.to_csv("datos/output/fechas.csv")

# Tiempo

- Tiempo
    - id_tiempo PK
    - id_estado_cielo (FK estado cielo)
    - id_municipio (FK municipio)
    - id_fecha (FK fecha)
    - temperatura
    - sensacion_temp
    - racha_max
    - precipitación
    - nieve
    - humedad
    - probabilidad precipitación
    - probabilidad nieve
    - probabilidad tormenta
    - Dirección viento

In [22]:
aemet.head(1)

,fecha,cielo,temp._(°c),sen._térmica_(°c),racha_máx._(km/h),precipitación_(mm),nieve_(mm),humedad_relativa_(%),prob._precip.__(%),prob._de_nieve_(%),prob._de_tormenta_(%),avisos,dirección_viento,velocidad_del_viento,localizacion_id
0,2024-08-29 12:00:00,Nubes altas,21,21,25.0,0.0,0,69,95,0,75,Riesgo,S,7,acebeda-la-id28001


### Obterner Índices
Primero vamos a obtener el índice del estado del cielo, voy a utilizar una función

In [23]:
df_cielo.head(1)
df_cielo_work = df_cielo
df_cielo_work.index = pd.RangeIndex(start=1,stop = 24,step=1)
df_cielo_work.reset_index(inplace=True)
df_cielo_work.head()

,index,estado_cielo
0,1,Nubes altas
1,2,Muy nuboso
2,3,Cubierto con tormenta y lluvia escasa
3,4,Nuboso
4,5,Cielo despejado


In [24]:
df_aemet = aemet
df_aemet = obtain_sky_state(df_aemet,df_cielo_work)

In [25]:
df_aemet.sample()

,fecha,cielo,indice_estado_cielo,temp._(°c),sen._térmica_(°c),racha_máx._(km/h),precipitación_(mm),nieve_(mm),humedad_relativa_(%),prob._precip.__(%),prob._de_nieve_(%),prob._de_tormenta_(%),avisos,dirección_viento,velocidad_del_viento,localizacion_id
2806,2024-08-30 16:00:00,Nubes altas,1,27,27,11.0,0.0,0,45,Riesgo,0,70,Sin riesgo,S,2,gascones-id28064


Lo mismo para la fecha

In [26]:
df_fecha_work = df_fechas
df_fecha_work.index = pd.RangeIndex(start=1,stop = 46,step=1)
df_fecha_work.reset_index(inplace=True)
df_fecha_work.head()

,index,fecha
0,1,2024-08-29 12:00:00
1,2,2024-08-29 13:00:00
2,3,2024-08-29 14:00:00
3,4,2024-08-29 15:00:00
4,5,2024-08-29 16:00:00


In [27]:
df_aemet2 = df_aemet
df_aemet2 = obtain_date_index(df_aemet2,df_fecha_work)
df_aemet2.sample()

c:\Users\ruipe\Desktop\Data Science\Semana 4\labs\laboratorio-modulo4-leccion03-SqlPython\src\SupportLab.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_to_modify["indice_fecha"] = df_to_modify["fecha"].map(diccionario)


,fecha,indice_fecha,cielo,indice_estado_cielo,temp._(°c),sen._térmica_(°c),racha_máx._(km/h),precipitación_(mm),nieve_(mm),humedad_relativa_(%),prob._precip.__(%),prob._de_nieve_(%),prob._de_tormenta_(%),avisos,dirección_viento,velocidad_del_viento,localizacion_id
4570,2024-08-30 20:00:00,33,Nubes altas,1,23,23,8.0,0.0,0,74,45,0,45,Sin riesgo,N,8,pedrezuela-id28108


In [28]:
df_aemet3 = df_aemet2
df_aemet3[["localizacion","id"]] = df_aemet3["localizacion_id"].str.rsplit("-",n=1,expand = True)
df_aemet3.sample()

C:\Users\ruipe\AppData\Local\Temp\ipykernel_2504\5655269.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aemet3[["localizacion","id"]] = df_aemet3["localizacion_id"].str.rsplit("-",n=1,expand = True)
C:\Users\ruipe\AppData\Local\Temp\ipykernel_2504\5655269.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aemet3[["localizacion","id"]] = df_aemet3["localizacion_id"].str.rsplit("-",n=1,expand = True)


,fecha,indice_fecha,cielo,indice_estado_cielo,temp._(°c),sen._térmica_(°c),racha_máx._(km/h),precipitación_(mm),nieve_(mm),humedad_relativa_(%),prob._precip.__(%),prob._de_nieve_(%),prob._de_tormenta_(%),avisos,dirección_viento,velocidad_del_viento,localizacion_id,localizacion,id
6702,2024-08-29 18:00:00,7,Nuboso,4,25,25,32.0,0.0,0,43,Riesgo,0,75,Riesgo,S,19,valdelaguna-id28157,valdelaguna,id28157


Ahora la localización id, que haremos un regex 

In [29]:
df_aemet3.drop(columns=["id","localizacion_id"],inplace=True)

In [30]:
df_aemet3.sample()

,fecha,indice_fecha,cielo,indice_estado_cielo,temp._(°c),sen._térmica_(°c),racha_máx._(km/h),precipitación_(mm),nieve_(mm),humedad_relativa_(%),prob._precip.__(%),prob._de_nieve_(%),prob._de_tormenta_(%),avisos,dirección_viento,velocidad_del_viento,localizacion
1199,2024-08-29 21:00:00,10,Nubes altas,1,17,17,5.0,0.0,0,68,Riesgo,0,60,Riesgo,SO,3,bustarviejo


### Ahora que ya tenemos el municipio bien usamos el df de municipios para darle su índice

In [31]:
df_municipios_work = municipios.T
df_municipios_work = df_municipios_work.reset_index()
df_municipios_work.drop(columns=[0,1,2],inplace= True)
df_municipios_work.index = pd.RangeIndex(start=1,stop = 177,step=1)
df_municipios_work.reset_index(inplace=True)


In [32]:
df_municipios_work = df_municipios_work.rename(columns={"level_0" : "index","index" : "municipio"})

In [33]:
df_aemet3 = obtain_municipio_index(df_aemet3,df_municipios_work)

In [34]:
condi = df_aemet3["indice_municipio"].isnull() == True
checkin = df_aemet3[condi]
checkin["localizacion"].value_counts()

localizacion
cabrera-la     44
leganes        44
molinos-los    44
Name: count, dtype: int64

### Descubrimos que municipios necesita 3 valores más: cabrera-la, leganes y molino-los

In [35]:
df_municipios_modif = municipios.T

In [36]:
df_municipios_modif.reset_index(inplace=True)

In [37]:
df_municipios_modif

,index,0,1,2
0,acebeda-la,41.0869583,-3.624399,La Acebeda
1,ajalvir,40.5342302,-3.4807818,Ajalvir
2,alameda-del-valle,40.9177178,-3.8438216,Alameda del Valle
3,alamo-el,40.2307296,-3.9905893,El Álamo
4,alcala-de-henares,40.4818396,-3.3644973,Alcalá de Henares
...,...,...,...,...
171,villar-del-olmo,40.3364465,-3.2355624,Villar del Olmo
172,villarejo-de-salvanes,40.1683582,-3.2738764,Villarejo de Salvanés
173,villaviciosa-de-odon,40.3573787,-3.9002334,Villaviciosa de Odón
174,villavieja-del-lozoya,41.0063622,-3.6716291,Villavieja del Lozoya


In [38]:
df_municipios_modif = df_municipios_modif.rename(columns={"index":"municipio sucio",0 : "latitud", 1:"longitud",2:"nombre municipio"})

In [39]:
df_municipios_modif.sample()

,municipio sucio,latitud,longitud,nombre municipio
38,chapineria,40.379164,-4.2097254,Chapinería


In [40]:
municipios = ["cabrera-la","leganes","molinos-los"]

In [41]:
df_municipios = geopy_search(municipios,df_municipios_modif)

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:01<00:00,  2.38it/s]


### Lo guardamos para luego, que ya está arreglado

In [42]:
df_municipios.to_csv("datos/output/fixed_municipios.csv")

In [43]:
df_municipios.index = pd.RangeIndex(start=1,stop = 180,step=1)

In [44]:
df_municipios.reset_index(inplace=True)

In [45]:
df_municipios = df_municipios.drop(columns=["latitud","longitud","nombre municipio"])

In [46]:
df_municipios

,index,municipio sucio
0,1,acebeda-la
1,2,ajalvir
2,3,alameda-del-valle
3,4,alamo-el
4,5,alcala-de-henares
...,...,...
174,175,villavieja-del-lozoya
175,176,zarzalejo
176,177,cabrera-la
177,178,leganes


In [47]:
df_municipios = df_municipios.rename(columns={"municipio sucio":"municipio"})

In [48]:
df_aemet3 = obtain_municipio_index(df_aemet3,df_municipios)

In [49]:
df_aemet3.sample()

,fecha,indice_fecha,cielo,indice_estado_cielo,temp._(°c),sen._térmica_(°c),racha_máx._(km/h),precipitación_(mm),nieve_(mm),humedad_relativa_(%),prob._precip.__(%),prob._de_nieve_(%),prob._de_tormenta_(%),avisos,dirección_viento,velocidad_del_viento,localizacion,indice_municipio
1556,2024-08-30 02:00:00,15,Nuboso,4,18,18,22.0,0.0,0,98,5,0,5,Sin riesgo,NE,15,casarrubuelos,35


In [50]:
df_aemet3 = df_aemet3.drop(columns=["fecha","cielo","avisos","localizacion","prob._precip.__(%)"])

In [51]:
df_aemet3.to_csv("datos/output/tabla_aemet.csv")

# Lugares
- Lugares
    - id_lugar PK
    - id_categoría (FK categorías)
    - id_municipio (FK municipio)
    - id_tiempo (FK tiempo)
    - nombre
    - dirección
    - distancia (al centro del municipio)